# Desafio 5

Neste desafio, vamos praticar sobre redução de dimensionalidade com PCA e seleção de variáveis com RFE. Utilizaremos o _data set_ [Fifa 2019](https://www.kaggle.com/karangadiya/fifa19), contendo originalmente 89 variáveis de mais de 18 mil jogadores do _game_ FIFA 2019.

> Obs.: Por favor, não modifique o nome das funções de resposta.

## _Setup_ geral

In [1]:
from math import sqrt

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as sct
import seaborn as sns
import statsmodels.api as sm
import statsmodels.stats as st
from sklearn.decomposition import PCA

from loguru import logger

In [2]:
# Algumas configurações para o matplotlib.
#%matplotlib inline

"""from IPython.core.pylabtools import figsize


figsize(12, 8)

sns.set()"""

'from IPython.core.pylabtools import figsize\n\n\nfigsize(12, 8)\n\nsns.set()'

In [3]:
fifa = pd.read_csv("fifa.csv")

In [4]:
columns_to_drop = ["Unnamed: 0", "ID", "Name", "Photo", "Nationality", "Flag",
                   "Club", "Club Logo", "Value", "Wage", "Special", "Preferred Foot",
                   "International Reputation", "Weak Foot", "Skill Moves", "Work Rate",
                   "Body Type", "Real Face", "Position", "Jersey Number", "Joined",
                   "Loaned From", "Contract Valid Until", "Height", "Weight", "LS",
                   "ST", "RS", "LW", "LF", "CF", "RF", "RW", "LAM", "CAM", "RAM", "LM",
                   "LCM", "CM", "RCM", "RM", "LWB", "LDM", "CDM", "RDM", "RWB", "LB", "LCB",
                   "CB", "RCB", "RB", "Release Clause"
]

try:
    fifa.drop(columns_to_drop, axis=1, inplace=True)
except KeyError:
    logger.warning(f"Columns already dropped")

## Inicia sua análise a partir daqui

#### Características dos dados

In [5]:
print(f'Linhas: {fifa.shape[0]} | Colunas: {fifa.shape[1]}')

Linhas: 18207 | Colunas: 37


In [6]:
fifa.head().T

,0,1,2,3,4
Age,31.0,33.0,26.0,27.0,27.0
Overall,94.0,94.0,92.0,91.0,91.0
Potential,94.0,94.0,93.0,93.0,92.0
Crossing,84.0,84.0,79.0,17.0,93.0
Finishing,95.0,94.0,87.0,13.0,82.0
HeadingAccuracy,70.0,89.0,62.0,21.0,55.0
ShortPassing,90.0,81.0,84.0,50.0,92.0
Volleys,86.0,87.0,84.0,13.0,82.0
Dribbling,97.0,88.0,96.0,18.0,86.0
Curve,93.0,81.0,88.0,21.0,85.0


In [7]:
print(f'Colunas presentes: {fifa.columns.values}')

Colunas presentes: ['Age' 'Overall' 'Potential' 'Crossing' 'Finishing' 'HeadingAccuracy'
 'ShortPassing' 'Volleys' 'Dribbling' 'Curve' 'FKAccuracy' 'LongPassing'
 'BallControl' 'Acceleration' 'SprintSpeed' 'Agility' 'Reactions'
 'Balance' 'ShotPower' 'Jumping' 'Stamina' 'Strength' 'LongShots'
 'Aggression' 'Interceptions' 'Positioning' 'Vision' 'Penalties'
 'Composure' 'Marking' 'StandingTackle' 'SlidingTackle' 'GKDiving'
 'GKHandling' 'GKKicking' 'GKPositioning' 'GKReflexes']


In [8]:
print(f'Tipos dos dados: {fifa.dtypes.unique()}')

Tipos dos dados: [dtype('int64') dtype('float64')]


#### Informações dos dados

In [9]:
fifa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18207 entries, 0 to 18206
Data columns (total 37 columns):
Age                18207 non-null int64
Overall            18207 non-null int64
Potential          18207 non-null int64
Crossing           18159 non-null float64
Finishing          18159 non-null float64
HeadingAccuracy    18159 non-null float64
ShortPassing       18159 non-null float64
Volleys            18159 non-null float64
Dribbling          18159 non-null float64
Curve              18159 non-null float64
FKAccuracy         18159 non-null float64
LongPassing        18159 non-null float64
BallControl        18159 non-null float64
Acceleration       18159 non-null float64
SprintSpeed        18159 non-null float64
Agility            18159 non-null float64
Reactions          18159 non-null float64
Balance            18159 non-null float64
ShotPower          18159 non-null float64
Jumping            18159 non-null float64
Stamina            18159 non-null float64
Strength       

#### Estatística dos dados

In [10]:
fifa.describe().T

,count,mean,std,min,25%,50%,75%,max
Age,18207.0,25.122206,4.669943,16.0,21.0,25.0,28.0,45.0
Overall,18207.0,66.238699,6.908930,46.0,62.0,66.0,71.0,94.0
Potential,18207.0,71.307299,6.136496,48.0,67.0,71.0,75.0,95.0
Crossing,18159.0,49.734181,18.364524,5.0,38.0,54.0,64.0,93.0
Finishing,18159.0,45.550911,19.525820,2.0,30.0,49.0,62.0,95.0
HeadingAccuracy,18159.0,52.298144,17.379909,4.0,44.0,56.0,64.0,94.0
ShortPassing,18159.0,58.686712,14.699495,7.0,54.0,62.0,68.0,93.0
Volleys,18159.0,42.909026,17.694408,4.0,30.0,44.0,57.0,90.0
Dribbling,18159.0,55.371001,18.910371,4.0,49.0,61.0,68.0,97.0
Curve,18159.0,47.170824,18.395264,6.0,34.0,48.0,62.0,94.0


#### Análise da consistência das váriáveis

In [11]:
data_missing = pd.DataFrame({'nomes': fifa.columns, 
                             'tipos': fifa.dtypes, 
                             'NA #': fifa.isna().sum(),
                             'NA %': (fifa.isna().sum() / fifa.shape[0]) * 100,
                             'size' : fifa.shape[0],
                             'uniques': fifa.nunique()
                            })

data_missing[data_missing['NA #'] > 0].set_index('nomes')

,tipos,NA #,NA %,size,uniques
nomes,,,,,
Crossing,float64,48,0.263635,18207,89
Finishing,float64,48,0.263635,18207,93
HeadingAccuracy,float64,48,0.263635,18207,91
ShortPassing,float64,48,0.263635,18207,85
Volleys,float64,48,0.263635,18207,87
Dribbling,float64,48,0.263635,18207,94
Curve,float64,48,0.263635,18207,89
FKAccuracy,float64,48,0.263635,18207,90
LongPassing,float64,48,0.263635,18207,84


#### Dropando valores nulos

In [12]:
fifa.dropna(inplace=True)

## Questão 1

Qual fração da variância consegue ser explicada pelo primeiro componente principal de `fifa`? Responda como um único float (entre 0 e 1) arredondado para três casas decimais.

In [13]:
def q1():
    pca = PCA(n_components=1).fit(fifa)
    return round(float(pca.explained_variance_ratio_), 3)

In [14]:
q1()

0.565

## Questão 2

Quantos componentes principais precisamos para explicar 95% da variância total? Responda como un único escalar inteiro.

In [15]:
def q2():
    pca = PCA(n_components=0.95).fit(fifa)
    return int(pca.n_components_)

In [16]:
q2()

15

## Questão 3

Quais são as coordenadas (primeiro e segundo componentes principais) do ponto `x` abaixo? O vetor abaixo já está centralizado. Cuidado para __não__ centralizar o vetor novamente (por exemplo, invocando `PCA.transform()` nele). Responda como uma tupla de float arredondados para três casas decimais.

In [17]:
x = [0.87747123,  -1.24990363,  -1.3191255, -36.7341814,
     -35.55091139, -37.29814417, -28.68671182, -30.90902583,
     -42.37100061, -32.17082438, -28.86315326, -22.71193348,
     -38.36945867, -20.61407566, -22.72696734, -25.50360703,
     2.16339005, -27.96657305, -33.46004736,  -5.08943224,
     -30.21994603,   3.68803348, -36.10997302, -30.86899058,
     -22.69827634, -37.95847789, -22.40090313, -30.54859849,
     -26.64827358, -19.28162344, -34.69783578, -34.6614351,
     48.38377664,  47.60840355,  45.76793876,  44.61110193,
     49.28911284
]

In [18]:
def q3():
    pca = PCA().fit(fifa)
    return tuple(pca.components_.dot(x)[:2].round(3))

In [19]:
q3()

(186.556, -6.592)

## Questão 4

Realiza RFE com estimador de regressão linear para selecionar cinco variáveis, eliminando uma a uma. Quais são as variáveis selecionadas? Responda como uma lista de nomes de variáveis.

In [20]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

In [21]:
def q4():
    target = fifa['Overall']
    X = fifa.drop(['Overall'], axis=1)

    rfe = RFE(estimator=LinearRegression(), n_features_to_select=5).fit(X, target)
    index_selected = rfe.get_support(indices=True)
    return list(X.columns[index_selected])

In [22]:
q4()

['Age', 'Potential', 'BallControl', 'Reactions', 'GKReflexes']